## Use Decision Tree for forcast IMOEX price
### Vol 2
#### 13.06.19

In [1]:
# connecting nessesary paceges
from pandas import read_csv as read
from pandas import DataFrame as DF
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from IPython.display import SVG
from IPython.display import display
from graphviz import Source
from IPython.display import HTML
style = "<style>svg{width:80% !important;height:80% !important;}</style>"
HTML(style)

In [2]:
# In this part used decision tree in dynamic
# Used time frame 10 sec
data = read('https://raw.githubusercontent.com/Aurilino/Python/master/Decision%20Tree%20IMOEX/IMOEX_10_sec.txt')
data.index = range(len(data))
data['MOEX_1'] = data.MOEX.shift(-1) # let's try to predict one step forvard
data['MOEX_1'] = DF(data['MOEX_1']).apply(lambda x: np.log(x) - np.log(x).shift(1))
data['MOEX_1'] = data['MOEX_1'].apply(lambda x: 1 if x > 0 else 0)
data = data.drop(['Time', 'MOEX'], axis=1)
data.dropna(inplace=True)
data.head()

,LKOH,SBER,GAZP,NVTK,MOEX_1
0,5169.0,234.22,235.34,1306.4,0
1,5168.0,234.25,235.40,1306.4,1
2,5170.5,234.31,235.67,1306.6,0
3,5171.5,234.40,235.65,1306.6,0
4,5171.5,234.28,235.56,1306.6,0


### Use decision tree in dynamics
#### Try to predict for up one step forward 

In [3]:
step = 100
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3)
start = 0
finish = start + step
result = list(data.MOEX_1)[:finish+2]
while finish <= len(data)-3:
    train = data.iloc[start:finish, :]
    x = train.drop('MOEX_1', axis=1)
    y = train.MOEX_1
    clf.fit(x, y)
    x_test = data.iloc[finish+1:finish+2, :].drop('MOEX_1',axis=1)
    result.append(int(clf.predict(x_test)))
    start += 1
    finish += 1
data['MOEX_pred'] = result
# get share of mistakes
data['diff'] = abs(data.MOEX_1 - data.MOEX_pred)
print('Share of mistakes: ', round(data['diff'].sum() / len(data['diff']) * 100, 2), '%',sep='')

Share of mistakes: 20.26%


## Summary
### As we can see, use strategy with use decision tree, we can predict near 80% future behavior of index IMOEX.